# 0.0. Step 0 - Imports

In [1]:
import math
import numpy as np
import pandas as pd
import inflection

import seaborn as sns

from numpy import int64
from matplotlib import pyplot as plt
from IPython.core.display import HTML
from IPython.display import Image

## 0.1. Helper Functions 

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [20, 12]
    plt.rcParams['font.size'] = 25
#    
    display(HTML( '<style>.container {width:100% !important;}</style>'))
#    pd.options.display.max_columns = None
#    pd.options.display.max_rows = None
#    pd.set_option('display.expand_frame_repr', False)
            
    sns.set() 

In [3]:
jupyter_settings ()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

# 1.0. Step 1 - Descrição dos Dados

## 1.1. Data

In [4]:
df_sales_raw = pd.read_csv ('data/train.csv', low_memory = ddd)
df_xxxxxxxx_raw = pd.read_csv ('data/xxxxxxxx.csv', low_memory = ddd)

#dd
df_raw = pd.dd (df_sales_raw, df_xxxxxxxx_raw, how='left', on='xxxxxxxx')

NameError: name 'ddd' is not defined

In [ ]:
df1 = df_raw.copy()

## 1.2. Rename Columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday', 'SchoolHoliday',
            'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth','CompetitionOpenSinceYear',
            'Promo2', 'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.3. Data Dimensions

In [ ]:
print('Number of Rows {}'.format(df1.shape[0]))
print('Number od Columns {}'.format(df1.shape[1]))

## 1.4. Data Types

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

## 1.5. Check NA

In [ ]:
df1.isna().sum()

## 1.6. Fillout NA

In [ ]:
#competition_distance 
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 2000000.0 if math.isnan (x) else (x))

#competition_open_since_month  
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year    
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1 )

#promo2_since_week              
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1 )

#promo2_since_year 
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1 )

#promo_interval
month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
df1['promo_interval'].fillna(0, inplace=True)
df1['month_map'] = df1['date'].dt.month.map(month_map)
df1['is_promo'] = df1[['promo_interval', 'month_map' ]].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',') else 0, axis=1)


## 1.7. Change Types

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int64)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int64)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int64)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int64)

## 1.8. Descriptive Statistical

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64','datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [ ]:
# central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - std, min, max, range, skew, Kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

In [ ]:
sns.kdeplot(df1['sales']);

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0] )

In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]
aux2 = df1[df1['sales'] > 0]

plt.subplot(131)
sns.boxplot(x='state_holiday', y='sales', data=aux1);

plt.subplot(132)
sns.boxplot(x='store_type', y='sales', data=aux2);

plt.subplot(133)
sns.boxplot(x='assortment', y='sales', data=aux2);

# 2.0. Step 2 - Feature Engineering

### 2.1. Mind Map Hypothesis

In [ ]:
Image('img/MindMapHypothesis.png')

### 2.2. Create Hypothesis

#### 2.2.1. Store Hypothesis

1.Lojas com número maior de funcionários deveriam vender mais.

2.Lojas com maior capacidade de estoque deveriam vender mais.

3.Lojas com maior porte deveriam vender mais.

4.Lojas com maior sortimentos deveriam vender mais.

5.Lojas com competidores mais próximos deveriam vender menos.

6.Lojas com competidores à mais tempo deveriam vendem mais.

#### 2.2.2. Product Hypothesis

**1**.Lojas que investem mais em Marketing deveriam vender mais.

**2**.Lojas com maior exposição de produto deveriam vender mais.

**3**.Lojas com produtos com preço menor deveriam vender mais.

**5**.Lojas com promoções mais agressivas ( descontos maiores ), deveriam vender mais.

**6**.Lojas com promoções ativas por mais tempo deveriam vender mais.

**7**.Lojas com mais dias de promoção deveriam vender mais.

**8**.Lojas com mais promoções consecutivas deveriam vender mais.

#### 2.2.3. Time Hypothesis (seasonality)

**1**.Lojas abertas durante o feriado de Natal deveriam vender mais.

**2**.Lojas deveriam vender mais ao longo dos anos.

**3**.Lojas deveriam vender mais no segundo semestre do ano.

**4**.Lojas deveriam vender mais depois do dia 10 de cada mês.

**5**.Lojas deveriam vender menos aos finais de semana.

**6**.Lojas deveriam vender menos durante os feriados escolares.

### 2.3. Final List Hypothesis

**1**.Lojas com maior sortimentos deveriam vender mais.

**2**.Lojas com competidores mais próximos deveriam vender menos.

**3**.Lojas com competidores à mais tempo deveriam vendem mais.

**4**.Lojas com promoções ativas por mais tempo deveriam vender mais.

**5**.Lojas com mais dias de promoção deveriam vender mais.

**6**.Lojas com mais promoções consecutivas deveriam vender mais.

**7**.Lojas abertas durante o feriado de Natal deveriam vender mais.

**8**.Lojas deveriam vender mais ao longo dos anos.

**9**.Lojas deveriam vender mais no segundo semestre do ano.

**10**.Lojas deveriam vender mais depois do dia 10 de cada mês.

**11**.Lojas deveriam vender menos aos finais de semana.

**12**.Lojas deveriam vender menos durante os feriados escolares.